In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:


import tensorflow as tf
import tensorflow_decision_forests as tfdf


In [ ]:


train_df = pd.read_csv("/kaggle/input/titanic/train.csv")
serving_df = pd.read_csv("/kaggle/input/titanic/test.csv")

train_df.head(10)



In [ ]:
def preprocess(df):
    df = df.copy()
    
    def normalize_name(x):
        return " ".join([v.strip(",()[].\"'") for v in x.split(" ")])
    
    def ticket_number(x):
        return x.split(" ")[-1]
        
    def ticket_item(x):
        items = x.split(" ")
        if len(items) == 1:
            return "NONE"
        return "_".join(items[0:-1])
    
    df["Name"] = df["Name"].apply(normalize_name)
    df["Ticket_number"] = df["Ticket"].apply(ticket_number)
    df["Ticket_item"] = df["Ticket"].apply(ticket_item)                     
    return df
    
preprocessed_train_df = preprocess(train_df)
preprocessed_serving_df = preprocess(serving_df)

preprocessed_train_df.head(5)

In [ ]:
input_features = list(preprocessed_train_df.columns)
input_features.remove("Ticket")
input_features.remove("PassengerId")
input_features.remove("Survived")
#input_features.remove("Ticket_number")

print(f"Input features: {input_features}")

In [ ]:
def tokenize_names(features, labels=None):
    """Divite the names into tokens. TF-DF can consume text tokens natively."""
    features["Name"] =  tf.strings.split(features["Name"])
    return features, labels

train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(preprocessed_train_df,label="Survived").map(tokenize_names)
serving_ds = tfdf.keras.pd_dataframe_to_tf_dataset(preprocessed_serving_df).map(tokenize_names)

In [ ]:
model = tfdf.keras.GradientBoostedTreesModel(
    verbose=0, # Very few logs
    features=[tfdf.keras.FeatureUsage(name=n) for n in input_features],
    exclude_non_specified_features=True, # Only use the features in "features"
    random_seed=1234,
)
model.fit(train_ds)

self_evaluation = model.make_inspector().evaluation()
print(f"Accuracy: {self_evaluation.accuracy} Loss:{self_evaluation.loss}")

In [ ]:
model = tfdf.keras.GradientBoostedTreesModel(
    verbose=0, # Very few logs
    features=[tfdf.keras.FeatureUsage(name=n) for n in input_features],
    exclude_non_specified_features=True, # Only use the features in "features"
    
    #num_trees=2000,
    
    # Only for GBT.
    # A bit slower, but great to understand the model.
    # compute_permutation_variable_importance=True,
    
    # Change the default hyper-parameters
    # hyperparameter_template="benchmark_rank1@v1",
    
    #num_trees=1000,
    #tuner=tuner
    
    min_examples=1,
    categorical_algorithm="RANDOM",
    #max_depth=4,
    shrinkage=0.05,
    #num_candidate_attributes_ratio=0.2,
    split_axis="SPARSE_OBLIQUE",
    sparse_oblique_normalization="MIN_MAX",
    sparse_oblique_num_projections_exponent=2.0,
    num_trees=2000,
    #validation_ratio=0.0,
    random_seed=1234,
    
)
model.fit(train_ds)

self_evaluation = model.make_inspector().evaluation()
print(f"Accuracy: {self_evaluation.accuracy} Loss:{self_evaluation.loss}")

In [ ]:
from sklearn.preprocessing import OneHotEncoder
k=list(train_df.columns)
from sklearn import preprocessing
from sklearn.metrics import mean_absolute_error
print(k)
#Import svm model
from sklearn import svm


enc = OneHotEncoder()

#train_df.drop(['Survived'], axis=1)
train_df['Embarked'] = train_df['Embarked'].fillna('C')

train_df = train_df.fillna(0)
#train_df.dropna() 
#train_df = train_df.reset_index(drop=True)
#train_df.dropna(axis=0, how='all', subset=['Ticket', 'PassengerId','Name','Sex','Cabin','Embarked'], inplace=True)
  
one_hot_encoded_data = pd.get_dummies(train_df, columns = ['Sex','Embarked'])


#one_hot_encoded_data=one_hot_encoded_data.dropna() 
#one_hot_encoded_data= one_hot_encoded_data.reset_index(drop=True)
#df.drop('a', inplace=True, axis=1)

#train_x = train_df
#label_encoder = preprocessing.LabelEncoder()
one_hot_encoded_data.drop(['Name'],inplace=True, axis=1)
one_hot_encoded_data.drop(['Ticket'],inplace=True, axis=1)
one_hot_encoded_data.drop(['Cabin'],inplace=True, axis=1)
  
# Encode labels in column 'species'.
#train_df['Ticket', 'PassengerId','Name','Sex','Cabin','Embarked']= label_encoder.fit_transform(train_df['Ticket', 'PassengerId','Name','Sex','Cabin','Embarked'])

y_train=one_hot_encoded_data['Survived']
y_train = y_train.fillna(0)
#y_train.dropna()
#y_train = y_train.reset_index(drop=True)

#X_train.columns = X_train.columns.astype(str)
one_hot_encoded_data.drop(['Survived'],inplace=True, axis=1)
#train_df = train_df.fillna(0)
X_train=one_hot_encoded_data
print('below is X_train')
  
print(X_train)
             

clf = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
def predict(a):
    
    a = a.fillna(0)
    one_hot_encoded_data = pd.get_dummies(a, columns = ['Sex','Embarked'])

    one_hot_encoded_data=one_hot_encoded_data.dropna() 
    one_hot_encoded_data = one_hot_encoded_data.reset_index(drop=True)
    one_hot_encoded_data.drop(['Name'],inplace=True, axis=1)
    one_hot_encoded_data.drop(['Ticket'],inplace=True, axis=1)
    one_hot_encoded_data.drop(['Cabin'],inplace=True, axis=1)
    #y_train=one_hot_encoded_data['Survived']


    #X_train.columns = X_train.columns.astype(str)
    #one_hot_encoded_data.drop(['Survived'], axis=1)
    X1_train=one_hot_encoded_data
    y_pred = clf.predict(X1_train)
    print(y_pred)
    df1=a['PassengerId'].values
    df1= pd.DataFrame(df1)
    
    y_pred= pd.DataFrame(y_pred)
    submission=pd.concat([df1, y_pred], axis=1, keys=['PassengerId', 'Survived'])
    #submission = df1.join(df2, how='outer', lsuffix='_df1', rsuffix='_df2')
    print('below is sub file')
    print(submission)
    submission.to_csv('submission.csv')
    
    #mean_absolute_error(, y_pred)
predict(serving_df)








In [ ]:
#clf.summary()